# Natural Language Embedded Programs (NLEP)

NLEP involves prompting a language model to create and execute a Python program to solve a user’s query, and then output the solution as natural language.

NLEPs are fully executable Python programs containing appropriate package importing, structured natural language representations of knowledge, function definitions for problem solving, and response printing.

from paper [Natural Language Embedded Programs for Hybrid Language Symbolic Reasoning](https://arxiv.org/pdf/2309.10814), 2024

An NLEP is a problem-solving template with four steps:
- First, the model calls the necessary packages, or functions, it will need to solve the task. 
- Step two involves importing natural language representations of the knowledge the task requires (like a list of U.S. presidents’ birthdays). 
- For step three, the model implements a function that calculates the answer. 
- And for the final step, the model outputs the result as a line of natural language with an automatic data visualization, if needed.

each NLEP contains four sections: (1) importing necessary libraries, (2) defining variables containing structured knowledge, (3) implementing problem-solving functions, and (4) printing the response in natural language.

In [52]:
question = "Which U.S. presidents elected after 1950 were born on a Wednesday?"

In [10]:
from openai import OpenAI
client = OpenAI(api_key="ollama", base_url="http://localhost:11434/v1")

completion = client.chat.completions.create(
    model="llama3.2:latest",
    messages=[{"role": "system", "content": "You are a helpful assistant. If you could not answer the question, then answer just with 'no answer possible'"},
              {"role": "user", "content": question}])

print(completion.choices[0].message)

ChatCompletionMessage(content='No answer possible', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [11]:
if completion.choices[0].message.content.lower() == 'no answer possible':
    print("reformulate the question:")
    reformulation = f"How would you answer the question: {question}. Let’s think step-by-step."
    
    completion = client.chat.completions.create(
    model="llama3.2:latest",
    messages=[{"role": "system", "content": "You are a helpful assistant. If you could not answer the question, then answer just with 'no answer possible'"},
              {"role": "user", "content": reformulation}])
        
    print(completion.choices[0].message) 

reformulate the question:
ChatCompletionMessage(content="To solve this, let's break down the problem into manageable steps:\n\nStep 1: Identify U.S. presidents elected under presidents and their respective birth dates.\n\nUnfortunately, I've been trained up to February 2023, so I don't have information about future elections or presidents. However, I can provide you with a list of U.S. presidents born before January 1950.\n\nIn a separate scenario, if I had access to more comprehensive data beyond my knowledge cutoff date (February 2023), here are the steps:\n\n- List all the presidents who won at least one general election since 1948.\n- Identify those who were born between 1927 and 1992 (to cover 1950 onwards).\n- Check if among them, there is any president born on a Wednesday.\n\nHowever, I have limited information from beyond my training data up to February 2023.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [53]:
#### Use code sniplets from LangCode ####
#
# see: https://github.com/luohongyin/LangCode/blob/main/server/nlep_server.py

fs_prompt = open('nlep_prompt.txt').read()

def construct_fspy_prompt(fs_prompt, inst_str, input_txt = 'None'):
    prompt = f'''{fs_prompt}\n\n### Instruction: {inst_str}\n### Input: {input_txt}\n### Python program:'''
    return prompt

def generate(fs_prompt, ques_str, model="llama3.2:latest"):
    
    prompt = construct_fspy_prompt(fs_prompt = fs_prompt, inst_str = ques_str, input_txt = 'None')

    #messages=[{"role": "system", "content": "You are a helpful assistant. If you could not answer the question, then answer just with 'no answer possible'"},
    #          {"role": "user", "content": prompt}]

    messages=[{"role": "user", "content": prompt}]
    
    completion = client.chat.completions.create(model = model, messages = messages, 
                                                temperature = 0.5, top_p = 1.0, max_tokens = 1024)

    gen_txt = completion.choices[0].message.content.replace('```python', '```')
    
    res_str = (prompt + gen_txt).split('### Python program:')[-1].strip()

    sections = res_str.split('```')
    if len(sections) > 1:
        ans_str = sections[1].strip()
    else:
        ans_str = sections[0].strip()
    
    return ans_str

In [63]:
#result = generate(fs_prompt, ques_str=question, model="llama3.2:latest")
#result = generate(fs_prompt, ques_str=question, model="qwen2.5-coder:14b")
#result = generate(fs_prompt, ques_str=question, model="gemma2:9b")
result = generate(fs_prompt, ques_str=question, model="qwt")

In [64]:
print(result)

presidents = [
    ("Dwight D. Eisenhower", "October 14, 1890"),
    ("John F. Kennedy", "May 29, 1917"),
    ("Lyndon B. Johnson", "August 27, 1908"),
    ("Richard Nixon", "January 9, 1913"),
    ("Gerald Ford", "July 14, 1913"),
    ("Jimmy Carter", "October 1, 1924"),
    ("Ronald Reagan", "February 6, 1911"),
    ("George H. W. Bush", "June 12, 1924"),
    ("Bill Clinton", "August 19, 1946"),
    ("George W. Bush", "July 6, 1946"),
    ("Barack Obama", "August 4, 1961"),
    ("Donald Trump", "June 14, 1946"),
]

born_on_wednesday = [president for president, birthdate in presidents if birthdate.split(", ")[0].weekday() == 2]

print(', '.join(born_on_wednesday))


In [34]:
import datetime
presidents = {
    "John F. Kennedy": datetime.datetime(1917, 5, 29),
    "Lyndon B. Johnson": datetime.datetime(1908, 8, 27),
    "Richard Nixon": datetime.datetime(1913, 1, 9),
    "Gerald Ford": datetime.datetime(1913, 7, 14),
    "Jimmy Carter": datetime.datetime(1924,10,1),
    "Ronald Reagan": datetime.datetime(1911,2,6),
    "George H.W. Bush": datetime.datetime(1924,6,12),
    "Bill Clinton": datetime.datetime(1946,8,19),
    "George W. Bush": datetime.datetime(1946,7,6),
    "Barack Obama": datetime.datetime(1961,8,4),
    "Donald Trump": datetime.datetime(1946,6,14),
    "Joe Biden": datetime.datetime(1942,11,20),
}
def find_presidents_born_on_wednesday(presidents):
    wednesdays = [day for day in presidents.values() if day.weekday() == 2]
    return [name for name, day in presidents.items() if day in wednesdays]
presidents_born_on_wednesday = find_presidents_born_on_wednesday(presidents)
if presidents_born_on_wednesday:
    print(f"The U.S. Presidents elected after 1950 who were born on a Wednesday are {', '.join(presidents_born_on_wednesday)}.")

The U.S. Presidents elected after 1950 who were born on a Wednesday are Jimmy Carter.
